In [2]:
# Download dependencies
import pandas as pd
import requests 
from bs4 import BeautifulSoup
import re

## Get the list of resources

In [5]:
list_of_resources = pd.read_csv('/Users/kenlam/Desktop/Data science/ML projects/RAG_resume/model/list_resources.csv')
list_of_resources.head()

,name,link,website,form
0,10 Resume Writing Tips To Help You Land a Posi...,https://www.indeed.com/career-advice/resumes-c...,Indeed,article
1,How to Write a Résumé That Stands Out,https://hbr.org/2014/12/how-to-write-a-resume-...,Harvard Business Review,article
2,18 Résumé Writing Tips to Help You Stand Out,https://www.wsj.com/articles/18-resume-writing...,WSJ,article
3,Words To Avoid and Include on a Resume,https://www.indeed.com/career-advice/resumes-c...,Indeed,article
4,Everything You Need To Know About Job Applicat...,https://www.indeed.com/career-advice/finding-a...,Indeed,article


### Divide the list into articles and youtube

In [8]:
list_of_articles = list_of_resources[list_of_resources['form'] == 'article']
list_of_yt = list_of_resources[list_of_resources['form'] == 'video']

## Gather articles from articles giving resume advice

In [12]:
# Get a list of urls
list_of_urls = list(list_of_articles['link'])

In [39]:
def detect_block(url):
    """Detects access to the article's content"""
    warning_phrases = ["You have been blocked", "Page not found"]

    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    content = soup.get_text(strip = True).lower()
    
    for phrase in warning_phrases:
        if phrase.lower() in content:
            return True
    return False

In [40]:
def extract_content(url):
    """Extracts the raw content of an article if didn't get block"""
    try:
        # If there's no warning messages
        if detect_block(url):
            print(f"Warning detected for {url}")
            return None
        
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        content = soup.get_text(strip = True)
        print(f"Successfully extract content for {url}")
        return content
    except requests.RequestException as e:
        print(f"Failed to scrape {url}: {e}")
        return None

In [41]:
def clean_text(content):
    """Cleans the text by removing unecessary characters like whitespaces"""
    cleaned_content = re.sub(r'\s+', ' ', content)  # Remove extra whitespace
    cleaned_content = re.sub(r'<.*?>', '', content)  # Remove HTML tags)
    return cleaned_content.strip()

In [44]:
url = list_of_urls[10]
content = extract_content(url)
cleaned_content = clean_text(content)
print(cleaned_content)

Successfully extract content for https://extendedstudies.ucsd.edu/news-and-events/division-of-extended-studies-blog/how-to-land-a-job-in-2024-resume-tips
12 Resume Best Practices from a Career Advisor: How to Make Your Resume Stand Out in 2024 | Continuing Education | UC San Diego Division of Extended StudiesSkip to Contentshopping_cartsearchtwitterinstagramfacebookgooglelinkedinyoutubemailthumbs-upphoneChat Bubbleenvelope iconLocation Pin IconDownload IconcalendarmobileNew Campus Location:As of June 17th, 2024, we have moved to our brand-new campus at 8980 Villa La Jolla Drive, directly across the street from UC San Diego. Student Services and International Programs are on-site and ready to assist students. For directions or general inquiries, please contact us at (858) 534-3400. We look forward to welcoming you as we usher in a new era of innovation to serve our vibrant San Diego community and beyond.View Additional InformationToggle main menu visibilityCollaboratePartner With UsCust

In [46]:
list_articles_content = []

for url in list_of_urls:
    content = extract_content(url)
    if content:
        cleaned_content = clean_text(content)
        list_articles_content.append(cleaned_content)

Warning detected for https://www.indeed.com/career-advice/resumes-cover-letters/10-resume-writing-tips
Successfully extract content for https://hbr.org/2014/12/how-to-write-a-resume-that-stands-out
Warning detected for https://www.wsj.com/articles/18-resume-writing-tips-to-get-you-noticed-11605107446
Warning detected for https://www.indeed.com/career-advice/resumes-cover-letters/words-to-avoid-and-include-on-a-resume
Warning detected for https://www.indeed.com/career-advice/finding-a-job/about-job-applications
Warning detected for https://www.indeed.com/career-advice/finding-a-job/completing-job-application-to-get-an-interview
Warning detected for https://www.indeed.com/career-advice/resumes-cover-letters/how-to-make-a-resume-with-examples
Warning detected for https://www.indeed.com/career-advice/resumes-cover-letters/best-resume-skills
Successfully extract content for https://www.linkedin.com/pulse/24-resume-tips-get-you-hired-2024-steph-cartwright-cprw-5z3kc/
Failed to scrape https:/

In [ ]:
# Save the outputs in a corpus.txt
with open("corpus.txt", 'w', encoding='utf-8') as f:
    for article in list_articles_content:
        f.write(article + '\n\n')

## Gather transcriptions from YouTube videos giving resume advice